<a href="https://colab.research.google.com/github/Inshalmunaf/Deep_Learning_Project1/blob/main/DeepLearning__Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samuelcortinhas/cats-and-dogs-image-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'cats-and-dogs-image-classification' dataset.
Path to dataset files: /kaggle/input/cats-and-dogs-image-classification


In [64]:
import os
import pandas as pd
import numpy as np
import cv2

In [65]:
Test_path = os.path.join(path,'test')
Train_path = os.path.join(path,'train')


In [78]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D , MaxPool2D ,Dense, Flatten , BatchNormalization , Dropout , RandomFlip ,RandomRotation , RandomZoom
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [67]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = Train_path,
    labels = 'inferred',
    label_mode ='int',
    batch_size = 32,
    image_size = (256,256)
)
test_ds = keras.utils.image_dataset_from_directory(
    directory = Test_path,
    labels = 'inferred',
    label_mode ='int',
    batch_size = 32,
    image_size = (256,256)
)

Found 557 files belonging to 2 classes.
Found 140 files belonging to 2 classes.


In [68]:
#Normalize
def process(image , label):
  image = tf.cast(image/255 , tf.float32)
  return image,label

test_ds = test_ds.map(process)
train_ds = train_ds.map(process)


In [93]:
# Model Building
model = Sequential()

#Augmentation
model.add(RandomFlip('horizontal' , input_shape=(256,256,3)))
model.add(RandomRotation(0.1))
model.add(RandomZoom(0.1))

model.add(Conv2D( 32 ,kernel_size=(3,3) , padding= 'valid' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2) , padding='valid' , strides = 2))

model.add(Conv2D( 64 ,kernel_size=(3,3) , padding= 'valid' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2) , padding='valid' , strides = 2))

model.add(Conv2D( 128 ,kernel_size=(3,3) , padding= 'valid' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2) , padding='valid' , strides = 2))

model.add(Conv2D( 256 ,kernel_size=(3,3) , padding= 'valid' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2) , padding='valid' , strides = 2))

model.add(Flatten())

model.add(Dense(256 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1 , activation = 'sigmoid'))

In [87]:
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_14 (RandomFlip)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_14              │ (None, 256, 256, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_14 (RandomZoom)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_46 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_47 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_48 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,193 (56.64 MB)

 Trainable params: 14,847,745 (56.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [97]:
lr_scheduler = ReduceLROnPlateau(monitor= 'val-accuracy' , factor = 0.2 ,patience = 3 , verbose = 1 )
model.compile(optimizer = Adam(learning_rate=0.0001) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [98]:
history = model.fit(train_ds , epochs= 15  , validation_data = test_ds , callbacks = [lr_scheduler])

Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 367ms/step - accuracy: 0.7869 - loss: 0.5263 - val_accuracy: 0.4714 - val_loss: 1.0263 - learning_rate: 1.0000e-04
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 282ms/step - accuracy: 0.7440 - loss: 0.7115 - val_accuracy: 0.5214 - val_loss: 1.4332 - learning_rate: 1.0000e-04
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - accuracy: 0.7124 - loss: 0.7438 - val_accuracy: 0.5500 - val_loss: 1.5022 - learning_rate: 1.0000e-04
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - accuracy: 0.7193 - loss: 0.6535 - val_accuracy: 0.4714 - val_loss: 1.8781 - learning_rate: 1.0000e-04
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.7485 - loss: 0.6022 - val_accuracy: 0.5000 - val_loss: 1.5709 - learning_rate: 1.0000e-04
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.7747 - loss: 0.4496 - val_accuracy: 0.5071 - val_loss: 1.4692 - learning_rate: 1.0000e-04
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 214ms/step - accuracy:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'] , color = 'red' ,label = 'training')
plt.plot(history.history['val_accuracy'] , color = 'blue' ,label = 'Validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'] , color = 'red' ,label = 'test')
plt.plot(history.history['val_loss'] , color = 'blue' ,label = 'Validation')
plt.legend()
plt.show()